https://openai.com/blog/introducing-text-and-code-embeddings  
https://platform.openai.com/docs/guides/embeddings

In [25]:
from dotenv import load_dotenv
import os
load_dotenv()  # .env 파일에서 환경 변수 로드
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')  # OPENAI_API_KEY 환경 변수 가져오기

In [26]:
import openai
import numpy as np # 벡터 연산을 위해 numpy 라이브러리를 임포트합니다.
openai.api_key = OPENAI_API_KEY

In [27]:
# 문서들을 불러옵니다.
documents = [
    "파이썬은 인터프리터 언어로 높은 생산성을 자랑하며, 데이터 분석 및 웹 개발에 활용되고 있다.",
    "자바는 객체 지향 프로그래밍 언어로, 안드로이드 애플리케이션 개발 및 서버 개발에 널리 사용되고 있다.",
    "자바스크립트는 웹 브라우저에서 동작하는 스크립팅 언어로, 웹 개발에 필수적인 언어이다.",
]

Similarity embedding

In [28]:
# 문서들에 대한 임베딩을 생성합니다.
doc_embeddings = []
for doc in documents:
    response = openai.Embedding.create(
        input=doc,
        engine="text-similarity-davinci-001"
    )
    print(response["data"][0]["embedding"])
    doc_embeddings.append(response["data"][0]["embedding"])

[-0.010719646699726582, 0.00015877287660259753, -0.011620350182056427, -0.012730373069643974, -0.005226620472967625, 0.01664399541914463, -0.00041070833685807884, -0.0013954569585621357, 0.001097336644306779, -0.006336642894893885, -0.002887644339352846, -0.009971174411475658, 0.005651600658893585, -0.0023469049483537674, 0.00018206352251581848, 0.002408748958259821, -0.006514246575534344, -0.016885029152035713, -0.0027829851023852825, 0.006067065987735987, -0.004706495441496372, -0.010389811359345913, 0.009793570265173912, -0.003694789484143257, 0.0026434394530951977, -0.0009823699947446585, 0.012419566512107849, 0.006295413710176945, -0.024242892861366272, -0.006273213308304548, -0.007738443091511726, 0.009355904534459114, -0.007243690080940723, -0.0021249002311378717, -0.0008111093193292618, 0.0028258003294467926, 0.012698657810688019, -0.013332956470549107, 0.009178300388157368, 0.011645722202956676, -0.003368125529959798, -0.0062066116370260715, 0.005753088276833296, -0.0028559295

In [29]:
# 쿼리에 대한 임베딩을 생성합니다.
query = "웹 개발에 사용되는 프로그래밍 언어는 무엇인가요?"
response = openai.Embedding.create(
    input=query,
    engine="text-similarity-davinci-001"
)
print(response["data"][0]["embedding"])
query_embedding = response["data"][0]["embedding"]

[-0.003343053860589862, 0.014108067378401756, -0.007999223656952381, -0.01790151372551918, -0.00990863423794508, 0.01845974661409855, -0.0045419856905937195, 0.0006117566372267902, 0.0012147908564656973, -0.00934405717998743, -0.007637640926986933, -0.0033811151515692472, -0.0015866818139329553, -0.0037775873206555843, 0.006172279827296734, 0.0023724897764623165, -0.002853014273568988, -0.0099022900685668, -0.007479052059352398, 0.012033725157380104, -0.004402427468448877, -0.007136499974876642, 0.0202359426766634, -0.007263371255248785, 0.003495299257338047, -0.003964722156524658, 0.019449342042207718, 0.006429193541407585, -0.01640443503856659, -0.01505960151553154, -0.0099022900685668, -0.0025088763795793056, 0.0007806538487784564, -0.004665685351938009, -0.005258807446807623, 0.008944413624703884, 0.010549332946538925, -0.00998475681990385, 0.002738830167800188, 0.0042438386008143425, 0.0007017558673396707, -0.0013361114542931318, 0.0007604337297379971, -0.005851930007338524, 0.007

In [30]:
# 문서들과 쿼리 사이의 코사인 유사도를 계산합니다.
cosine_similarities = []
for doc_embedding in doc_embeddings:
    cosine_similarity = np.dot(query_embedding, doc_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(doc_embedding))
    cosine_similarities.append(cosine_similarity)

In [31]:
# 가장 유사한 문서를 찾습니다.
max_index = np.argmax(cosine_similarities)
most_similar_document = documents[max_index]

In [32]:
# 결과를 출력합니다.
print("The most similar document to the query is:")
print(cosine_similarities[max_index])
print('   query : ' + query)
print('document : ' + most_similar_document)

The most similar document to the query is:
0.845709344008339
   query : 웹 개발에 사용되는 프로그래밍 언어는 무엇인가요?
document : 자바스크립트는 웹 브라우저에서 동작하는 스크립팅 언어로, 웹 개발에 필수적인 언어이다.


In [33]:
openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages= [{"role": "user", "content":most_similar_document + query}]
)

<OpenAIObject chat.completion id=chatcmpl-72IHPjPpUggEHPhFrzl1nBR4oHtAT at 0x13668661040> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "\uc6f9 \uac1c\ubc1c\uc5d0 \uc0ac\uc6a9\ub418\ub294 \ud504\ub85c\uadf8\ub798\ubc0d \uc5b8\uc5b4\ub294 \ub2e4\uc591\ud558\uc9c0\ub9cc, HTML, CSS, JavaScript\uac00 \uac00\uc7a5 \uae30\ubcf8\uc801\uc778 \uac83\uc73c\ub85c \uc54c\ub824\uc838 \uc788\uc2b5\ub2c8\ub2e4. \uadf8 \uc678\uc5d0\ub3c4 PHP, Python, Ruby \ub4f1 \ub2e4\uc591\ud55c \uc5b8\uc5b4\uac00 \uc0ac\uc6a9\ub418\uae30\ub3c4 \ud569\ub2c8\ub2e4.",
        "role": "assistant"
      }
    }
  ],
  "created": 1680781279,
  "id": "chatcmpl-72IHPjPpUggEHPhFrzl1nBR4oHtAT",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 73,
    "prompt_tokens": 86,
    "total_tokens": 159
  }
}

In [34]:
# # 문서들을 불러옵니다.
# documents = [
#     "한국의 전통 음식 중 한가지는 김치이며, 배추와 고추장을 사용해 발효시킨 음식이다.",
#     "삼겹살은 돼지의 등심에서 얻은 고기로, 한국에서 인기 있는 바베큐 음식이다.",
#     "떡볶이는 떡과 어묵, 양파 등의 재료를 고추장과 설탕으로 조리한 한국의 대표적인 길거리 음식이다.",
# ]

In [35]:
# # 쿼리에 대한 임베딩을 생성합니다.
# query = "한국에서 대표적인 길거리 음식은 무엇인가요?"
# response = openai.Embedding.create(
#     input=query,
#     engine="text-search-davinci-query-001"
# )
# query_embedding = response["data"][0]["embedding"]

Text search embedding

In [36]:
# # 문서들에 대한 임베딩을 생성합니다.
# doc_embeddings = []
# for doc in documents:
#     response = openai.Embedding.create(
#         input=doc,
#         engine="text-search-davinci-doc-001"
#     )
#     print(response["data"][0]["embedding"])
#     doc_embeddings.append(response["data"][0]["embedding"])

In [37]:
# # 쿼리에 대한 임베딩을 생성합니다.
# query = "웹 개발에 사용되는 프로그래밍 언어는 무엇인가요?"
# response = openai.Embedding.create(
#     input=query,
#     engine="text-search-davinci-query-001"
# )
# print(response["data"][0]["embedding"])
# query_embedding = response["data"][0]["embedding"]